In [1]:
# Import all the libraries
import numpy as np
import pandas as pd
from datetime import datetime
import pdb
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [2]:
# Create the entropy function

def entropy(y):
    N= len(y)
    s= (y==1).sum()
    if s==0 or N==s:
        return 0
    p1=float(s/N)
    p0=1-p1 
    ent = -p1*np.log2(p1) -p0*np.log2(p0)
    return ent


In [3]:
# get the dat from train
def get_data():
    df= pd.read_csv('/Users/sunpreet/Downloads/train.csv')
    data =df.as_matrix()
    np.random.shuffle(data)
    X=data[:,1:]/255
    Y=data[:,0]
    return X,Y


In [4]:
# Create a decision tree class

class TreeNode:
    
    def __init__(self,current_depth=0,max_depth=None):
        self.current_depth = current_depth
        self.max_depth= max_depth
        
        
                    
    def fit(self,X,Y):  
        
        if len(Y)== 1 or len(set(Y)) == 1:
            self.col = None
            self.split = None
            self.left = None
            self.right = None
            self.prediction = Y[0]
        else:
            D=X.shape[1]
            cols = range(D)
            max_ig = 0
            best_split= None
            best_col=None
            for col in cols:
                ig,split = self.find_split(X,Y,col)
                if ig > max_ig:
                    max_ig = ig
                    best_split= split
                    best_col = col
                    
            if max_ig == 0:
                self.col = None
                self.split = None
                self.left = None
                self.right = None
                self.prediction = np.round(Y.mean())
            else:
                self.col=best_col
                self.split=best_split
                
                if self.current_depth==self.max_depth:
                    self.left = None
                    self.right = None
                    self.prediction = [
                        np.round(Y[X[:,best_col] < self.split].mean()),
                        np.round(Y[X[:,best_col] >= self.split].mean()),
                    ]

                else:
                    left_idx = (X[:,best_col] < best_split)
                    Xleft = X[left_idx]
                    Yleft = Y[left_idx]
                    self.left = TreeNode(self.current_depth + 1,self.max_depth)
                    self.left.fit(Xleft,Yleft)
                    
                    right_idx = (X[:,best_col] >= best_split)
                    Xright = X[right_idx]
                    Yright = Y[right_idx]
                    self.right = TreeNode(self.current_depth + 1,self.max_depth)
                    self.right.fit(Xright,Yright) 
    
    def find_split(self,X,Y,col):
            x_values=X[:,col]
            sort_indx = np.argsort(x_values)
            x_values=x_values[sort_indx]
            y_values=Y[sort_indx]
            
            boundaries = np.nonzero(y_values[:-1]!=y_values[1:])[0]
            best_split=None
            max_ig =0
            
            for i in boundaries:
                split = (x_values[i] + x_values[i+1]) / 2
                ig=self.find_ig(x_values,y_values,split)
                if ig > max_ig:
                    max_ig=ig
                    best_split=split
                    
            return max_ig,best_split
    
  
    def find_ig(self,X,Y,split):
            y0=Y[X<split]
            y1=Y[X>=split]
            N=len(Y)
            if len(y0)==0 or len(y0)== N:
                return 0
            p0 = len(y0) / N
            p1= 1- p0
            return entropy(Y)-p0*entropy(y0)-p1*entropy(y1)
  
        
    def predict_one(self,X):
            if self.col is not None and self.split is not None:
                feature=X[self.col]
                if feature < self.split:
                    if self.left:
                        p=self.left.predict_one(X)
                    else:
                        p=self.prediction[0]
                else:
                    if self.right:
                           p=self.right.predict_one(X)
                    else:
                        p= self.prediction[1]
            else:
                p= self.prediction
            return p
        
    def predict(self,X):
            N= len(X)
            p=np.zeros(N) 
            for i in range(N):
                p[i]= self.predict_one(X[i])
            return p

                                  

In [5]:
# Create a wrapper class

class DecisionTree:
    def __init__(self,max_depth=None):
        self.max_depth=max_depth
    def fit(self,X,Y):
        self.root=TreeNode(max_depth=self.max_depth)
        self.root.fit(X,Y)
    def predict(self,X):
        return self.root.predict(X)
    def score(self,X,Y):
        P=self.predict(X)
        return np.mean(Y==P)
    

    

In [6]:
if __name__=='__main__':
    X,Y=get_data()
    # only take 0s and 1s since we're doing binary classification
    idx = np.logical_or(Y == 0, Y == 1)
    X = X[idx]
    Y = Y[idx]
    
    # split the data
    Ntrain = len(Y)/2
    Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
    Xtest, Ytest = X[Ntrain:], Y[Ntrain:]
    
    #model = DecisionTree()
        
    model = DecisionTree(max_depth=7)
    t0 = datetime.now()
    model.fit(Xtrain, Ytrain)
    print ("Training time:", (datetime.now() - t0))

    t0 = datetime.now()
    print ("Train accuracy:", model.score(Xtrain, Ytrain))
    print ("Time to compute train accuracy:", (datetime.now() - t0))

    t0 = datetime.now()
    print ("Test accuracy:", model.score(Xtest, Ytest))
    print ("Time to compute test accuracy:", (datetime.now() - t0))
    

/Users/sunpreet/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/sunpreet/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Training time: 0:00:32.991454
Train accuracy: 1.0
Time to compute train accuracy: 0:00:00.015888
Test accuracy: 0.994101633394
Time to compute test accuracy: 0:00:00.015820


In [7]:
# Repeat the same with sklearn library
model1 = DecisionTreeClassifier()
X,Y=get_data()
#idx = np.logical_or(Y == 0, Y == 1)
#X = X[idx]
#Y = Y[idx]
Xtrain,Xtest,Ytrain,Ytest =train_test_split(X,Y,test_size=0.5)
t0 = datetime.now()
model1.fit(Xtrain, Ytrain)
print ("Training time:", (datetime.now() - t0))

prediction=model1.predict(Xtest)

t0 = datetime.now()
print ("Train accuracy:", model1.score(Xtrain, Ytrain))
print ("Time to compute train accuracy:", (datetime.now() - t0))

t0 = datetime.now()
print ("Test accuracy:", model1.score(Xtest, Ytest))
print ("Time to compute test accuracy:", (datetime.now() - t0))
 
print(metrics.classification_report(Ytest,prediction))

    

Training time: 0:00:05.888268
Train accuracy: 1.0
Time to compute train accuracy: 0:00:00.053995
Test accuracy: 0.836238095238
Time to compute test accuracy: 0:00:00.045917
             precision    recall  f1-score   support

          0       0.89      0.91      0.90      2050
          1       0.91      0.94      0.92      2339
          2       0.82      0.80      0.81      2097
          3       0.80      0.77      0.79      2172
          4       0.81      0.84      0.82      2019
          5       0.78      0.76      0.77      1895
          6       0.87      0.87      0.87      2080
          7       0.86      0.88      0.87      2211
          8       0.78      0.76      0.77      2035
          9       0.80      0.81      0.81      2102

avg / total       0.84      0.84      0.84     21000



In [8]:
# import random Forest Classifier
from sklearn.ensemble import RandomForestClassifier


In [9]:
# See if we get any better results from RandomForest
RFC= RandomForestClassifier()
X,Y=get_data()
#idx = np.logical_or(Y == 0, Y == 1)
#X = X[idx]
#Y = Y[idx]
Xtrain,Xtest,Ytrain,Ytest =train_test_split(X,Y,test_size=0.5)
t0 = datetime.now()
RFC.fit(Xtrain, Ytrain)
print ("Training time:", (datetime.now() - t0))

prediction=RFC.predict(Xtest)

t0 = datetime.now()
print ("Train accuracy:", RFC.score(Xtrain, Ytrain))
print ("Time to compute train accuracy:", (datetime.now() - t0))

t0 = datetime.now()
print ("Test accuracy:", RFC.score(Xtest, Ytest))
print ("Time to compute test accuracy:", (datetime.now() - t0))
 
print(metrics.classification_report(Ytest,prediction))

    


Training time: 0:00:01.306761
Train accuracy: 0.998857142857
Time to compute train accuracy: 0:00:00.132667
Test accuracy: 0.92880952381
Time to compute test accuracy: 0:00:00.123623
             precision    recall  f1-score   support

          0       0.95      0.98      0.96      2100
          1       0.96      0.97      0.97      2303
          2       0.90      0.93      0.92      2102
          3       0.89      0.90      0.90      2111
          4       0.92      0.94      0.93      2032
          5       0.92      0.89      0.90      1910
          6       0.96      0.95      0.96      2052
          7       0.94      0.93      0.94      2218
          8       0.92      0.88      0.90      2060
          9       0.93      0.89      0.91      2112

avg / total       0.93      0.93      0.93     21000

